# RivHealth: A River Health Platform.

### "Safeguarding Waterways through Data-Driven Insights."

![RivHealth Image](https://cdn.discordapp.com/attachments/1224661831981994044/1227979228776235109/rivhealth-1.jpeg?ex=662a5fc6&is=6617eac6&hm=162f88c07a1187b94613afac2de5ed05c547901be43484306a85d9425450defa&)

# Project Description:

River health has declined in the UK and worldwide. There is considerable public anger towards water companies for sewage discharges. While serious, these account for only about 6-8% of the issue, with pesticides and fertilizers having a greater impact, according to some estimates. Water companies have lost control of the narrative. Meanwhile, local action groups and citizen scientists possess useful data and insight. This project presents an opportunity for Sand Tech to develop a common national platform for sharing data on river health, which will factually inform the debate and stimulate evidence-based solutions.

# Problem statement

Our mission is to become the trusted source of accurate insights in the water industry. We've noticed a lack of understanding among stakeholders about river health and its impact. This includes issues such as sewage discharges, agricultural runoff, misconnected sewers, illegal dumping, and industrial waste. To tackle this, we're developing a platform to offer a comprehensive view of river health and pinpoint pollution sources. Our aim is to empower stakeholders, including water companies, river conservation trusts, environmental agencies, farmers, and local authorities, with data-backed insights for informed decision-making and policy development.

# Objective & Methodologies

This project aims to develop a platform that integrates utility discharge data, citizen science data, and environmental/stakeholder groups' data and utilizes artificial intelligence (AI) and machine learning (ML) to provide meaningful insights into river health. The platform will empower stakeholders with evidence-based information to address the challenges facing river health in the UK and improve public understanding of river ecosystems.

* Data acquisition through open data APIs, sensor data feeds, and citizen science portals.
* Data storage using AWS, a cloud-based platform
* Data cleaning, transformation, and feature engineering techniques using Python.
* Machine learning model development using tools like scikit-learn or TensorFlow (I have to ask the Data Scientists to confirm)
* Geospatial Analysis and data integration with tools like QGIS, PostGIS or ArcGIS.
* Interactive visualization platforms using Streamlit

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Import packages and  libraries</a>

<a href=#two>2. Data Loading</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Feature Engineering</a>

<a href=#five>5. Conclusion</a>

<a id="two"></a>

## 1. Importing Packages

<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description: Importing Packages ⚡                                                                                                    |
| :--------------------------------------------------------------------------------------------------------------------------------------- |
| In this section we imported the libraries

---


* Importing necessary libraries and modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import dotenv
import datetime
import folium
import pyproj
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
from io import StringIO
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
warnings.filterwarnings('ignore')


<a id="two"></a>

## 2. Loading the Data

<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description: Loading the data ⚡                                                          |
| :------------------------------------------------------------------------------------------- |
| In this we loaded in the data. |

---

In [ ]:
# Load and Locate the .env file
dotenv_path = '../.env'
dotenv.load_dotenv(dotenv_path)

# Get AWS credentials from environment variables
ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID")
SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
S3_BUCKET_NAME = os.environ.get("S3_BUCKET_NAME")

# Create an s3 client
s3_client = boto3.client("s3", aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

# Function to read file from S3
def read_s3_file(bucket_name, filepath):
    """
    Returns:
        a pandas dataframe

    Args:
        bucket_name (str): 
            Name of the s3 bucket

        filepath (str): 
            Path to the file in the s3 bucket 
            e.g. "datasets/raw_data/uk_environmental_agency_data/water_quality/individual_sampling_points/TH-PCHE0004.csv"
    """
    
    s3_object = s3_client.get_object(Bucket=bucket_name, Key=filepath)
    data = s3_object['Body'].read().decode('utf-8')
    return pd.read_csv(StringIO(data))

**Keys For the Datasets:**

* wq: water quality
* rl: river level
* rfl: river flow level
* rain: total rainfall
* rtct: River Thame Conservation Trust
* tw: Thames Water Company
* da: Discharge Alerts
* dcs: Discharge Current Status
* long: Long format
* wide: Wide format
* invert_obj: Invertebrate Objects (Animals)

* Load the dataset and perform initial exploration.
* Display first few rows of the dataset

In [ ]:
# Preprocessed data
wq = "datasets/river_thame_preprocessed_data/EA_thame_sampling_points_water_quality_data.csv"
rfl = "datasets/river_thame_preprocessed_data/river_thame_hydrology_stations_river_flow_level_wide_data.csv"
rl = "datasets/river_thame_preprocessed_data/river_thame_hydrology_stations_river_level_wide_data.csv"
rain = "datasets/river_thame_preprocessed_data/river_thame_hydrology_stations_total_rainfall_wide_data.csv"
rtct = "datasets/river_thame_preprocessed_data/RTCT_data.csv"
tw_da = "datasets/river_thame_preprocessed_data/Thames_Water_DischargeAlerts_River-Thame.csv"
tw_dcs = "datasets/river_thame_preprocessed_data/Thames_Water_DischargeCurrentStatus_River-Thame.csv"
invert_obj_metrics_ = "datasets/raw_data/uk_environmental_agency_data/invertebrate_object/INV_OPEN_DATA_METRICS_2024-04-08.csv"
invert_obj_site_ = "datasets/raw_data/uk_environmental_agency_data/invertebrate_object/INV_OPEN_DATA_SITE_2024-04-08.csv"
invert_obj_taxa_ = "datasets/raw_data/uk_environmental_agency_data/invertebrate_object/INV_OPEN_DATA_TAXA_2024-04-08.csv"

In [ ]:
# Load only preprocessed data so it does not impact memory
wq_wide = read_s3_file(S3_BUCKET_NAME, wq)
rfl_wide = read_s3_file(S3_BUCKET_NAME, rfl)
rl_wide = read_s3_file(S3_BUCKET_NAME, rl)
rain_wide = read_s3_file(S3_BUCKET_NAME, rain)
rtct_data = read_s3_file(S3_BUCKET_NAME, rtct)
tw_da_data = read_s3_file(S3_BUCKET_NAME, tw_da)
tw_dcs_data = read_s3_file(S3_BUCKET_NAME, tw_dcs)
invert_obj_metrics = read_s3_file(S3_BUCKET_NAME, invert_obj_metrics_)
invert_obj_site = read_s3_file(S3_BUCKET_NAME, invert_obj_site_)
invert_obj_taxa = read_s3_file(S3_BUCKET_NAME, invert_obj_taxa_)

<a id="four"></a>

## 3. Exploratory Data Analysis (EDA)

<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description: Exploratory data analysis ⚡                                                             |
| :------------------------------------------------------------------------------------------------------- |
| In this section, we performed an in-depth analysis for the data we loaded. |

---


### 3.1. Water Quality.

* Start by examining the dataset's structure, beginning with a review of the first five rows.

In [ ]:
wq_wide.head(5)

In [ ]:
wq_wide.shape

####  NaN Values

* It appears that there are numerous NaN values present. We will generate an image to visualize the rows containing these NaN values.

In [ ]:
# Calculate NaN counts in each column of wq_wide
nan_counts = wq_wide.isnull().sum()

# Plotting the NaN counts
# Create a bar plot
plt.figure(figsize=(10, 6))
nan_counts.plot(kind='barh', color='teal')
plt.xlabel('NaN Counts')
plt.ylabel('Columns')
plt.title('NaN Counts in Each Column of wq_wide')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest count at the top
plt.show()

**Working With NaN values**

* We have a lot of NaN values. Let us replace the NaN values with the mean values. The mean represents the central tendency of the data, making it a natural choice for imputation if we want to maintain the overall average of the dataset.

#### Data Type

* Before replacing missing values, we'll check the data types of each column to determine if they contain "Int" or "Float" values, enabling us to use statistical methods for imputation.

In [ ]:
# Check the data types of each column
data_type = wq_wide.dtypes

# Print the data types
print(data_type)


####  NaN Values

* Replace the NaN cells with the Mean of the column.

In [ ]:
columns_to_fill = ["ammoniacal_nitrogen", "bod_atu", "conductivity", "iron", "nitrogen_oxidised",
                   "unionised_ammonia", "nitrate", "nitrite", "dissolved_oxygen_%_saturation",
                   "orthophosphate", "dissolved_oxygen_O2", "Phosphate", "solids_suspended_at_105C",
                   "water_temperature_in_celsius", "ph"]

for column in columns_to_fill:
    # Fill NaN values in each column with the mean of that column
    wq_wide[column] = wq_wide[column].fillna(wq_wide[column].mean())

In [ ]:
wq_wide.head(5)

#### Statistical Summary

* After addressing the missing values by imputation, we proceed to analyze the statistical summary to gain insights into the dataset's attributes.

In [ ]:
for column in columns_to_fill:
    wq_wide[column] = wq_wide[column].fillna(wq_wide[column].mean())

# Select only numerical columns excluding 'northing' and 'easting'
numerical_columns = wq_wide.select_dtypes(include=['float64', 'int64']).drop(columns=['northing', 'easting'])

# Calculate summary statistics
summary_stats = {
    'count': numerical_columns.count(),
    'mean': numerical_columns.mean(),
    'std': numerical_columns.std(),
    'min': numerical_columns.min(),
    '25%': numerical_columns.quantile(0.25),
    '50%': numerical_columns.quantile(0.5),
    '75%': numerical_columns.quantile(0.75),
    'max': numerical_columns.max()
}

# Create a DataFrame from the dictionary
summary_stats_df = pd.DataFrame(summary_stats)

# Print the DataFrame
print(summary_stats_df.to_markdown())

**Sample Date Time Column**

* It seems that the datetime column is an object data type, we will be changing it to DateTime data type to avoid Limited Functionality and Loss of DateTime Features.

* This code will convert the 'sample_date_time' column to datetime data type in the DataFrame wq_wide.

In [ ]:
wq_wide['sample_date_time'] = pd.to_datetime(wq_wide['sample_date_time'])

**Converting Eastely and Northely Coodinates to Longitutes and Latitudes**

* Now to get an overview image of the River and the sampling points along the river so that we can start analysing our data indepth.

In [ ]:
# Define the projection system for BNG (British National Grid) and WGS84 (standard latitude and longitude)
bng = pyproj.Proj(init='epsg:27700')  # BNG projection
wgs84 = pyproj.Proj(init='epsg:4326')  # WGS84 projection (standard latitude and longitude)

# Function to convert easting and northing to longitude and latitude
def convert_easting_northing_to_lon_lat(easting, northing):
    lon, lat = pyproj.transform(bng, wgs84, easting, northing)
    return lon, lat

# Apply the conversion function to each row in the DataFrame
wq_wide['longitude'], wq_wide['latitude'] = convert_easting_northing_to_lon_lat(wq_wide['easting'], wq_wide['northing'])


In [ ]:
# View the DataFrame with longitude and latitude columns
print(wq_wide[['longitude', 'latitude']])


#### Map of the Sample Catchment - The Thame Operational Catchment including River Thame and its Tributaries 

In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[wq_wide['latitude'].mean(), wq_wide['longitude'].mean()], zoom_start=10)

# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)


In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[wq_wide['latitude'].mean(), wq_wide['longitude'].mean()], zoom_start=10)

# Define sampling sites with different colors
special_sites = ['TH-PTAE0064', 'TH-PTAE009', 'TH-PTAE0021', 'TH-PTAE0223', 'TH-PTAE0212',
                'TH-PTAE0024', 'TH-PTAR0028', 'TH-PTAR0020', 'TH-PTAR0021', 'TH-PTAR0048',
                'TH-PTAR0112', 'TH-PTAR0132', 'TH-PTAR0030', 'TH-PTAR0100', 'TH-PTAR0052',
                'TH-PTAR0022', 'TH-RSN0914']

# Add markers for each unique latitude and longitude point
for index, row in wq_wide.iterrows():
    # Check if the coordinates are unique
    if (row['latitude'], row['longitude']) not in unique_coordinates:
        # Create a popup with the sampling_point_notation
        popup = folium.Popup(row['sampling_point_notation'], parse_html=True)
        
        # Check if the sampling point is special
        if row['sampling_point_notation'] in special_sites:
            # Add marker with custom color
            folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='darkblue')).add_to(m)
        else:
            # Add marker with default color
            folium.Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)
        
        # Add the coordinates to the set
        unique_coordinates.add((row['latitude'], row['longitude']))


# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)

# Display the map
m


* The map shows the catchment area, river flow, and sampling points.
* Focus will be on sampling points nearest to the River Thame for thorough analysis.
* Data from these points will be analyzed over time and compared to identify variations.

**Sampling points include:**

* TH-PTAE0064,
TH-PTAE009,
TH-PTAE0021,
TH-PTAE0223,
TH-PTAE0212

* TH-PTAE0024,
TH-PTAR0028,
TH-PTAR0020,
TH-PTAR0021,
TH-PTAR0048,
TH-PTAR0112

* TH-PTAR0132,
TH-PTAR0030,
TH-PTAR0100,
TH-PTAR0052,
TH-PTAR0022,
TH-RSM0914

* Let us filter the data for select sampling points along the river to reduce the map load time.

In [ ]:
# Define sampling sites with different colors
special_sites = ['TH-PTAE0064', 'TH-PTAE009', 'TH-PTAE0021', 'TH-PTAE0223', 'TH-PTAE0212',
                'TH-PTAE0024', 'TH-PTAR0028', 'TH-PTAR0020', 'TH-PTAR0021', 'TH-PTAR0048',
                'TH-PTAR0112', 'TH-PTAR0132', 'TH-PTAR0030', 'TH-PTAR0100', 'TH-PTAR0052',
                'TH-PTAR0022', 'TH-RSN0914']

filtered_wq = wq_wide[wq_wide['sampling_point_notation'].isin(special_sites)]

print(filtered_wq.shape)

* Now let us view the sampling points along the river on the map.

###  Thame Catchment Area Map with Water Quality Sampling Points

In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[wq_wide['latitude'].mean(), wq_wide['longitude'].mean()], zoom_start=10)

# Add markers for each unique latitude and longitude point
for index, row in filtered_wq.iterrows():
    # Check if the coordinates are unique
    if (row['latitude'], row['longitude']) not in unique_coordinates:
        # Create a popup with the sampling_point_notation
        popup = folium.Popup(row['sampling_point_notation'], parse_html=True)
        
        # Check if the sampling point is special
        if row['sampling_point_notation'] in special_sites:
            # Add marker with custom color
            folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='darkblue')).add_to(m)
        else:
            # Add marker with default color
            folium.Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)
        
        # Add the coordinates to the set
        unique_coordinates.add((row['latitude'], row['longitude']))


# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)

# Display the map
m

* **TH-PTAE0064 (ROWSHAM STW)** is the first sampling point we will work on since it is more upstream and it is located where the river starts. 

* **TH-PTAR0028 (THAME AT STONE BRIDGE, AYLESBURY)** will be the second sampling point to observe. After TH-PTAE0028 (THAME AT STONE BRIDGE, AYLESBURY).
*  **TH-PTAR0020 (THAME ABOVE EYTHROPE LAKE)** will be the third sampling point to observe, and this will continue till we get to the last sampling point along the River Thame where it meets River Thames at DORCHESTER STW (TH-PTAE0024).

#### Determinant Against Sampling Points

* We will generate separate time series plots for each determinant against the various sampling points. This process involves iterating over each determinant and creating a distinct plot for each one.

In [ ]:
# Threshold values for determinants with units
thresholds = {
    "ammoniacal_nitrogen": {"lower": 0, "upper": 1, "unit": "mg/l"},
    "bod_atu": {"lower": 0, "upper": 4, "unit": "mg/l"},
    "iron": {"lower": 0, "upper": 50, "unit": "ug/l"},
    "ph": {"lower": 6.5, "upper": 8.5, "unit": "phunits"},
    "nitrogen_oxidised": {"lower": 0, "upper": 2, "unit": "mg/l"},
    "nitrate": {"lower": 0, "upper": 10, "unit": "mg/l"},
    "nitrite": {"lower": 0, "upper": 1, "unit": "mg/l"},
    "phosphate": {"lower": 0, "upper": 0.1, "unit": "mg/l"},
    "orthophosphate": {"lower": 0, "upper": 0.1, "unit": "mg/l"},
    "water_temperature_in_celsius": {"lower": 10, "upper": 25, "unit": "degrees Celsius"},
    "dissolved_oxygen_%_saturation": {"lower": 80, "upper": 120, "unit": "%"},
    "dissolved_oxygen_O2": {"lower": 5, "upper": 10, "unit": "mg/l"},
    "unionised_ammonia": {"lower": 0, "upper": 0.5, "unit": "mg/l"},
    "solids_suspended_at_105C": {"lower": 0, "upper": 50, "unit": "mg/l"},
    "conductivity": {"lower": 100, "upper": 800, "unit": "us/cm"}
}

# List of sampling points and their labels
sampling_points = {
    'TH-PTAE0064': 'ROWSHAM STW',
    'TH-PTAE0021': 'CUDDINGTON STW',
    'TH-PTAE0223': 'THE OLD FISHERMAN PUBLIC HOUSE STW :SHAB',
    'TH-PTAE0212': 'WATERSTOCK GOLF CLUB STW :WATERSTOCK',
    'TH-PTAE0024': 'DORCHESTER STW',
    'TH-PTAR0028': 'THAME AT STONE BRIDGE, AYLESBURY',
    'TH-PTAR0020': 'THAME ABOVE EYTHROPE LAKE',
    'TH-PTAR0021': 'THAME AT CUDDINGTON BRIDGE',
    'TH-PTAR0048': 'CHEARSLEY BROOK ABOVE THAME, CHEARSLEY',
    'TH-PTAR0112': 'LASHLAKE STREAM ABOVE SCOTSGROVE BROOK',
    'TH-PTAR0132': 'CUTTLE BROOK AT THAME',
    'TH-PTAR0030': 'THAME AT WHEATLEY BRIDGE',
    'TH-PTAR0100': 'DENTON BROOK AT CHIPPINGHURST MANOR, LIT',
    'TH-PTAR0052': 'CHALGROVE BROOK AT CHISELHAMPTON BRIDGE',
    'TH-PTAR0022': 'THAME AT DORCHESTER BRIDGE',
    'TH-RSM0914': 'OPPOSITE NEWINGTON HOUSE'
}

# Define determinants
determinants = [
    "ammoniacal_nitrogen", "bod_atu", "conductivity", "iron", "nitrogen_oxidised",
    "unionised_ammonia", "nitrate", "nitrite", "dissolved_oxygen_%_saturation",
    "orthophosphate", "dissolved_oxygen_O2","solids_suspended_at_105C",
    "water_temperature_in_celsius", "ph"
]

# Create a separate plot for each determinant
for determinant in determinants:
    # Create empty figure
    fig = px.line(title=f'Time Series Plot of {determinant} for Different Sampling Points',
                labels={'sample_date_time': 'Date', 'value': 'Value'},
                template='plotly_white')
    
    # Add lines for each sampling point
    for sample_point, label in sampling_points.items():
        # Filter the dataframe for the current sample point
        sample_data = wq_wide[wq_wide['sampling_point_notation'] == sample_point]
        
        # Plot the data
        fig.add_scatter(x=sample_data['sample_date_time'], y=sample_data[determinant],
                        mode='lines', name=label)
    
    # Add threshold lines
    fig.add_hline(y=thresholds[determinant]["lower"], line_dash="dash", line_color="red",
                annotation_text=f"Lower Limit ({thresholds[determinant]['lower']} {thresholds[determinant]['unit']})", 
                annotation_position="bottom right")
    fig.add_hline(y=thresholds[determinant]["upper"], line_dash="dash", line_color="red",
                annotation_text=f"Upper Limit ({thresholds[determinant]['upper']} {thresholds[determinant]['unit']})", 
                annotation_position="top right")
    
    # Highlight regions exceeding the upper limit in red
    fig.update_traces(hoverinfo='skip')
    if not sample_data.empty:
        fig.update_layout(shapes=[
            dict(type='rect', xref='paper', yref='y', 
                x0=0, x1=1, y0=thresholds[determinant]["upper"], y1=max(sample_data[determinant]),
                fillcolor='rgba(255, 0, 0, 0.2)', layer='below', line_width=0),
            dict(type='rect', xref='paper', yref='y', 
                x0=0, x1=1, y0=min(sample_data[determinant]), y1=thresholds[determinant]["lower"],
                fillcolor='rgba(0, 0, 255, 0.2)', layer='below', line_width=0)
        ])
    
    # Customize layout
    fig.update_layout(xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d', title_text='Date'),
                    yaxis=dict(title=f'{determinant} ({thresholds[determinant]["unit"]})'),
                    hovermode='x unified',
                    xaxis_rangeslider_visible=True,
                    plot_bgcolor='#B2DFDB')  # Set background color
    
    # Show the plot
    fig.show()


**This is a brief summary of the determinands:**

* **Solids, Suspended at 105 C Sources:** livestock, bank erosion, land cultivation, storm runoff, dredging.
* **Dissolved Oxygen (% Saturation):** Adequate levels of dissolved oxygen are crucial for supporting aquatic life, as many organisms, including fish and macroinvertebrates, require oxygen to survive.
* **BOD : 5 Day ATU:** Biochemical Oxygen Demand (BOD) is a test which measures the dissolved oxygen (DO) content of a sample of water, the sample is then incubated under controlled conditions (20Â°C) for 5 days and the DO content re-measured. The drop in DO is the BOD. ATU means that allyl thiourea (ATU) has been added to suppress nitrification during the course of the test.
* **Nitrogen, Total Oxidized (as N):** TON is an essential parameter in water quality assessment because it reflects the overall nitrogen pollution in aquatic systems. Nitrogen compounds can originate from various sources such as agricultural runoff, wastewater discharges, industrial effluents, and atmospheric deposition.
* **Orthophosphate, reactive as P:** Orthophosphate is a form of phosphorus that can contribute to eutrophication and algal blooms in rivers and lakes. Excessive orthophosphate levels can lead to nutrient pollution and degrade water quality.
* **Iron:** Elevated iron levels can occur naturally or as a result of industrial activities. While some organisms require iron, excessive iron concentrations can be toxic to aquatic life and stain water bodies.
* **Nitrate:** Elevated nitrate levels can indicate contamination from agricultural runoff or wastewater, which can lead to eutrophication, algal blooms, and harm to aquatic ecosystems.
* **pH:** pH levels can affect the solubility of metals and nutrients in water, as well as the health of aquatic organisms. Extremes in pH (too acidic or too alkaline) can be detrimental to aquatic life.
* **Nitrite:** Nitrite can be toxic to aquatic organisms, particularly at high concentrations, and can interfere with the oxygen-carrying capacity of hemoglobin in fish.

**Outcome.**

* From the above visual we can see that a few of the determinants keep changing in some sampling points and stable in the rest but we have a few that a  re constantly unstable.

#### Determinants Parameters

The next step is to understand the parameters for each determinant, categorizing values as within limits, above the limit, or below the limit, to assess river health. Below are the defined parameters for each determinant, indicating what constitutes a healthy river.

#### Ammoniacal Nitrogen as N (mg/l)
    Within limits: 0 - 1 mg/l
    Above the required limit: > 1 mg/l
    Below the limits: < 0 mg/l
#### BOD : 5 Day ATU (mg/l)
    Within limits: 0 - 4 mg/l
    Above the required limit: > 4 mg/l
    Below the limits: < 0 mg/l
#### Iron (ug/l)
    Within limits: 200 - 1000 ug/l
    Above the required limit: > 1000 ug/l
    Below the limits: < 200 ug/l
#### pH (phunits)
    Within limits: 6.5 - 8.5 phunits
    Above the required limit: > 8.5 phunits
    Below the limits: < 6.5 phunits
#### Nitrogen, Total Oxidised as N (mg/l)
    Within limits: 0 - 2 mg/l
    Above the required limit: > 2 mg/l
    Below the limits: < 0 mg/l
#### Nitrate as N (mg/l)
    Within limits: 0 - 10 mg/l
    Above the required limit: > 10 mg/l
    Below the limits: < 0 mg/l
#### Nitrite as N (mg/l)
    Within limits: 0 - 1 mg/l
    Above the required limit: > 1 mg/l
    Below the limits: < 0 mg/l
#### Phosphate :- {TIP} (mg/l)
    Within limits: 0 - 0.1 mg/l
    Above the required limit: > 0.1 mg/l
    Below the limits: < 0 mg/l
#### Orthophosphate, reactive as P (mg/l)
    Within limits: 0 - 0.1 mg/l
    Above the required limit: > 0.1 mg/l
    Below the limits: < 0 mg/l
#### Temperature of Water (cel)
    Within limits: 10 - 25 degrees Celsius
    Above the required limit: > 25 degrees Celsius
    Below the limits: < 10 degrees Celsius
#### Oxygen, Dissolved, % Saturation (%)
    Within limits: 80 - 120%
    Above the required limit: > 120%
    Below the limits: < 80%
##### Oxygen, Dissolved as O2 (mg/l)
    Within limits: 5 - 10 mg/l
    Above the required limit: > 10 mg/l
    Below the limits: < 5 mg/l
#### Ammonia un-ionised as N (mg/l)
    Within limits: 0 - 0.5 mg/l
    Above the required limit: > 0.5 mg/l
    Below the limits: < 0 mg/l
#### Solids, Suspended at 105 C (mg/l)
    Within limits: 0 - 50 mg/l
    Above the required limit: > 50 mg/l
    Below the limits: < 0 mg/l
#### Conductivity at 25 C (us/cm)
    Within limits: 100 - 800 us/cm
    Above the required limit: > 800 us/cm
    Below the limits: < 100 us/cm

### 3.2. Thames Water Discharge Alerts for River Thame

The next dataset we're examining pertains to discharge alerts from Thames Water Company, the UK's largest water and wastewater company. Our objective is to pinpoint the discharge locations and analyze the frequency of these discharges.

* Firstly lets look at the Discharge Alerts.

In [ ]:
tw_da_data.head(10)

* Now we are going to convert the X and Y into Longitude and Latitudes.

In [ ]:
tw_da_data.head()

#### Plot showing discharge locations and Sampling points

* Let's create a plot showing the discharge locations alongside the identified sampling points near the River Thame and in the catchment.

In [ ]:
# Define the projection system for BNG (British National Grid) and WGS84 (standard latitude and longitude)
bng = pyproj.Proj(init='epsg:27700')  # BNG projection
wgs84 = pyproj.Proj(init='epsg:4326')  # WGS84 projection (standard latitude and longitude)

# Function to convert X and Y to longitude and latitude
def convert_X_Y_to_lon_lat(X, Y):
    lon, lat = pyproj.transform(bng, wgs84, X, Y)
    return lon, lat

# Apply the conversion function to each row in the DataFrame
tw_da_data['longitude'], tw_da_data['latitude'] = convert_X_Y_to_lon_lat(tw_da_data['X'], tw_da_data['Y'])


* Let's create a plot showing the discharge locations alongside the identified sampling points near the River Thame and in the catchment.

In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[wq_wide['latitude'].mean(), wq_wide['longitude'].mean()], zoom_start=10)

# Add markers for each unique latitude and longitude point
for index, row in filtered_wq.iterrows():
    # Check if the coordinates are unique
    if (row['latitude'], row['longitude']) not in unique_coordinates:
        # Create a popup with the sampling_point_notation
        popup = folium.Popup(row['sampling_point_notation'], parse_html=True)
        
        # Check if the sampling point is special
        if row['sampling_point_notation'] in special_sites:
            # Add marker with custom color
            folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='darkblue')).add_to(m)
        else:
            # Add marker with default color
            folium.Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)
        
        # Add the coordinates to the set
        unique_coordinates.add((row['latitude'], row['longitude']))

# Filter the DataFrame to include only unique discharge points
unique_discharge_points = tw_da_data[tw_da_data['AlertType'].isin(['Start', 'Stop'])].drop_duplicates(subset=['latitude', 'longitude'])

# Add markers for each unique discharge point
for index, row in unique_discharge_points.iterrows():
    # Create a popup with the alert type and datetime
    popup = folium.Popup(f"Alert Type: {row['LocationName']}, DateTime: {row['DateTime']}", parse_html=True)
    
    # Add marker with red color for discharge points
    folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='red')).add_to(m)


# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)

# Display the map
m


* From the visual, we can spot around 25 discharge points scattered across the map. Some are near the river, while others are close to the sampling points along the river. Now, let's zoom in on the discharge point that releases the most water. This will help us understand its importance in the river system and what it means for water quality and how we manage wastewater.

#### Bar Graph of the frequency of discharges at different locations

In [ ]:
# Dividing the frequency of each unique location by two
location_frequency = tw_da_data['LocationName'].value_counts() / 2

# Plotting the bar graph
plt.figure(figsize=(12, 6), facecolor='#B2DFDB')  # Setting the figure size and background color
bars = location_frequency.plot(kind='bar', color='#325d79')  # Creating the bar plot with specified color
bars.set_facecolor('#B2DFDB')  # Setting the face color of the bars to match the background
plt.title('Frequency of Discharges by Location', color='teal')  # Setting the title
plt.xlabel('Location Name', color='teal')  # Setting the x-axis label
plt.ylabel('Frequency of Discharge', color='teal')  # Setting the y-axis label
plt.xticks(rotation=45, ha='right', color='teal')  # Rotating and aligning x-axis labels
plt.yticks(fontsize=12, color='teal')  # Setting the font size and color of y-axis ticks
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Adding horizontal grid lines
plt.tight_layout()  # Adjusting layout
plt.show()  # Displaying the plot


* Based on the Frequency of Discharges by Location visual, it's evident that Chinnor, Stone, Worminghall, and Wingrave have experienced the highest discharges over the past three years. Consequently, our focus will be on examining the sampling points in proximity to these areas, namely TH-PTAE0020 (Chinnor), TH-PTAE0071 (Stone), TH-PTAE0092 (Worminghall), and TH-PTAR0087 (Wingrave). Our objective is to assess whether these discharge activities have had any discernible impact on the determinants observed at these sampling points. By analyzing the determinants in these areas, we aim to gain insights into the potential effects of discharge on water quality parameters and environmental conditions, thereby informing targeted interventions and management strategies to address any identified issues.

#### Determinant Chart, with each plot comparing the four sampling points

In [ ]:
# Threshold values for determinants with units
thresholds = {
    "ammoniacal_nitrogen": {"lower": 0, "upper": 1, "unit": "mg/l"},
    "bod_atu": {"lower": 0, "upper": 4, "unit": "mg/l"},
    "solids_suspended_at_105C": {"lower": 0, "upper": 50, "unit": "mg/l"}
}

# Sampling points and their labels
sampling_points = {
    'TH-PTAE0020': 'Chinnor',
    'TH-PTAE0071': 'Stone',
    'TH-PTAE0092': 'Worminghall',
    'TH-PTAR0087': 'Wingrave'
}

# Define colors for the line plots
colors = {'Chinnor': 'grey', 'Stone': 'pink'}

# Define determinants
determinants = [
    "ammoniacal_nitrogen", "bod_atu", "solids_suspended_at_105C",
]

# Filter data for the range of interest (from 2022 to current date)
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime.now()
wq_wide_filtered = wq_wide[(wq_wide['sample_date_time'] >= start_date) & (wq_wide['sample_date_time'] <= end_date)]

# Create a separate plot for each determinant
for determinant in determinants:
    # Create empty figure
    fig = px.line(title=f'Time Series Plot of {determinant} for Different Sampling Points',
                  labels={'sample_date_time': 'Date', 'value': f'Value ({thresholds[determinant]["unit"]})'},
                  template='plotly_white')
    
    # Add lines for each sampling point
    for sample_point, label in sampling_points.items():
        # Filter the dataframe for the current sample point
        sample_data = wq_wide_filtered[wq_wide_filtered['sampling_point_notation'] == sample_point]
        
        # Plot the data for the current determinant with different colors
        if label in colors:
            fig.add_scatter(x=sample_data['sample_date_time'], y=sample_data[determinant],
                            mode='lines', name=label, line=dict(color=colors[label]))
        else:
            fig.add_scatter(x=sample_data['sample_date_time'], y=sample_data[determinant],
                            mode='lines', name=label)
    
    # Add threshold lines
    fig.add_hline(y=thresholds[determinant]["lower"], line_dash="dash", line_color="red",
                  annotation_text=f"Lower Limit ({thresholds[determinant]['lower']} {thresholds[determinant]['unit']})", 
                  annotation_position="bottom right")
    fig.add_hline(y=thresholds[determinant]["upper"], line_dash="dash", line_color="red",
                  annotation_text=f"Upper Limit ({thresholds[determinant]['upper']} {thresholds[determinant]['unit']})", 
                  annotation_position="top right")
    
    # Customize layout with background color
    fig.update_layout(xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d'),
                      yaxis=dict(title=f'Value ({thresholds[determinant]["unit"]})'),
                      hovermode='x unified',
                      xaxis_rangeslider_visible=True,
                      plot_bgcolor='#B2DFDB')  # Set background color
    
    # Show the plot
    fig.show()

* The visual analysis reveals that several determinants, namely ammoniacal nitrogen, BOD (biochemical oxygen demand), and suspended solids at 105°C, exhibit notable variations and occasionally surpass established thresholds. These fluctuations indicate potential impacts of sewage discharge on the river's water quality. Elevated levels of these determinants suggest contamination and reduced water quality, highlighting the influence of sewage discharge on the river's ecological health. Such findings underscore the need for effective management strategies to mitigate the adverse effects of sewage discharge on water quality and safeguard the integrity of the river ecosystem.

* This code generate separate time series plots for each determinant, with each plot comparing the four sampling points.

In [ ]:
# Filter the DataFrame for specific columns
tw_da_data = tw_da_data[['LocationName', 'AlertType', 'DateTime']]

# Convert the 'DateTime' column to datetime objects
tw_da_data['DateTime'] = pd.to_datetime(tw_da_data['DateTime'])

# Replace string values in the 'AlertType' column
tw_da_data['AlertType'] = tw_da_data['AlertType'].replace({'Start': 34, 'Offline start': 34, 'Stop': 30, 'Offline stop': 30})


In [ ]:
# Filter the DataFrame for Aylesbury data and select specific columns
aylesbury_data = tw_da_data[tw_da_data['LocationName'] == 'Aylesbury'][['LocationName', 'AlertType', 'DateTime']]

In [ ]:
# Convert the 'DateTime' column to datetime objects
aylesbury_data['DateTime'] = pd.to_datetime(aylesbury_data['DateTime'])

In [ ]:
# Replace string values in the 'AlertType' column
aylesbury_data['AlertType'] = aylesbury_data['AlertType'].replace({'Start': 34, 'Offline start': 34, 'Stop': 30, 'Offline stop': 30})

### 3.3. River Thame Hydrology Stations Total Rainfall Wide Data

* We're moving on to evaluate how rainfall variations affect river health indicators such as water quality, sedimentation rates, and aquatic biodiversity. Our aim is to understand whether increased rainfall leads to runoff pollution or erosion, influencing the overall ecological balance of the river ecosystem. To begin this analysis, we'll examine the current state of our data.

In [ ]:
rain_wide.head(5)

* To handle missing data, we will impute all NaN values with 0 under the assumption that these instances represent days when rainfall data was not captured due to absence of rain. This approach allows for a uniform treatment of missing values, facilitating the analysis by considering non-recorded rainfall as equivalent to zero rainfall for the respective days.

In [ ]:
# replacing all NaN values with 0
rain_wide = rain_wide.fillna(0)

# Display the first five rows of the DataFrame after replacing NaN values
print(rain_wide.head(5))


* Looks like all the NaN values are replaced, lets then see the shape then also check the count of NaN values.

In [ ]:
rain_wide.shape

In [ ]:
nan_count = rain_wide.isna().sum()
print(nan_count)

#### Mean Rainfall for 3 Sampling points
* Next step is to visualise the data. The following tasks and aims to visualize the mean rainfall over the years for the three different locations.

In [ ]:
# Convert 'date' column to datetime if it's not already
rain_wide['date'] = pd.to_datetime(rain_wide['date'])

# Extract year from the date
rain_wide['year'] = rain_wide['date'].dt.year

# Group by year and calculate the mean
yearly_mean = rain_wide.groupby('year').mean()

# Plotting
plt.figure(figsize=(12, 6), facecolor='#B2DFDB')  # Setting the figure size and background color

colors = ['#7DA0CA', '#052659', '#BBA9FF']  # Custom color scheme
locations = ['Aylesbury', 'Dancers End', 'Wheatley']
for i, column in enumerate(['aylesbury_total_rainfall', 'dancers_end_total_rainfall', 'wheatley_total_rainfall']):
    plt.bar(yearly_mean.index + (i - 1) * 0.2, yearly_mean[column], width=0.2, label=locations[i], color=colors[i])

plt.xlabel('Year', fontsize=12)
plt.ylabel('Mean Rainfall', fontsize=12)
plt.title('Mean Rainfall by Year and Location', fontsize=14)
plt.xticks(yearly_mean.index, fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10)
plt.grid(True)

# Customizing background and grid lines
plt.gca().set_facecolor('#B2DFDB')  # Light gray background
plt.gca().tick_params(axis='both', colors='teal')  # Change color of tick marks

plt.tight_layout()
plt.show()


* This visualization helps in understanding the long-term trends and variations in rainfall for the three different locations. By plotting the mean rainfall over the years, it provides insights into how rainfall patterns have evolved and differed across various regions, but from this we can say that through the years the amount of rain is not constant but Dances End.

#### Relationship Between Rainfall and Discharge

* Combining discharge timelines with a line graph of rainfall helps us see how they relate. By putting them together on the same timeline, we can easily spot any patterns or connections between when it rains and when the discharge changes. This visual setup lets us quickly see if there's a link between rainfall and discharge levels. It's like putting pieces of a puzzle together to understand how rain affects the sewage discharge.

In [ ]:
# Filter the dataframe for the Aylesbury area and all dates
Aylesbury_filtered_bar = aylesbury_data

# Plot the data for Aylesbury bar chart
fig = go.Figure()

# Add bar chart trace
fig.add_trace(go.Bar(
    x=Aylesbury_filtered_bar['DateTime'],
    y=Aylesbury_filtered_bar['AlertType'],
    name='Discharge',
    marker_color='#673AB7',
    marker_line_color='#673AB7',  # Set bar outline color
    marker_line_width=1,  # Set bar outline width
    width=8  # Adjust the width of the bars
))

# Filter the dataframe for the Aylesbury area and dates from 2022 to current date
Aylesbury_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Add line chart trace
fig.add_trace(go.Scatter(
    x=Aylesbury_filtered_line['date'],
    y=Aylesbury_filtered_line['aylesbury_total_rainfall'],
    mode='lines',
    name='Rainfall',
    line=dict(color='#7DA0CA')
))

# Customize layout
fig.update_layout(
    title='Timeline of Events and Rainfall for Aylesbury',
    xaxis=dict(title='Date/Time', type='date'),
    yaxis=dict(title='Discharge / Rainfall'),
    hovermode='x unified',
    xaxis_rangeslider_visible=True,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Add zoom tool to the toolbar
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the plot
fig.show()


* The visual depiction underscores a notable correlation between rainfall and subsequent sewage discharge, suggesting a direct influence of precipitation on sewage system dynamics. This relationship likely arises from rainwater infiltration into aging sewer systems, compounded by urbanization-induced surface runoff. Such conditions can overwhelm sewer networks designed to handle both wastewater and stormwater, resulting in heightened sewage discharge during rainfall events.

### 3.4. River Thame Hydrology Stations River Flow Level Wide

* Lets look at how the dataset looks. 

In [ ]:
rfl_wide.head(5)

#### NaN Values

* It looks like we have some NaN values on the data set, we will be replace them with the mean on each sample column.  

In [ ]:
nan_count = rfl_wide.isna().sum()
print(nan_count)

* Since we have the mean river flow having zero NaN values we are then going to drop the oth two columns and keep the mean river flow.

In [ ]:
# Drop the columns 'wheatley_max_river_flow_level' and 'wheatley_min_river_flow_level'
rfl_wide = rfl_wide.drop(columns=['wheatley_max_river_flow_level', 'wheatley_min_river_flow_level'])


In [ ]:
rfl_wide.head()

* Now that our data appears to be well-organized, our next step is to create visuals that illustrate the influence of river flow on the river, as well as identify other factors that impact river flow.

#### Donut chart to visualize the average river flow level by season

In [ ]:
# Convert the 'date' column to datetime format
rfl_wide['date'] = pd.to_datetime(rfl_wide['date'])

# Define the seasons
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Extract the month from the date
rfl_wide['month'] = rfl_wide['date'].dt.month

# Map the month to the corresponding season
rfl_wide['season'] = rfl_wide['month'].apply(get_season)

# Calculate the average river flow level for each season
seasonal_avg_flow = rfl_wide.groupby('season')['wheatley_mean_river_flow_level'].mean()

# Plot the donut chart with percentages outside the pie chart
fig, ax = plt.subplots(figsize=(8, 8))
wedges, texts, autotexts = ax.pie(seasonal_avg_flow, labels=seasonal_avg_flow.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'], wedgeprops=dict(width=0.4, edgecolor='teal', linewidth=3), textprops={'fontsize': 12, 'color': 'teal'}, radius=0.71)

# Set edge color of each wedge to match the line color
for wedge in wedges:
    wedge.set_edgecolor('teal')

plt.title('Average River Flow Level by Season', color='teal')  # Change color of title
plt.ylabel('')  # Remove y-axis label

# Set background color
fig.patch.set_facecolor('#B2DFDB')

plt.show()

**The outcomes of the average river flow level by season indicate interesting patterns that can provide insights into factors affecting river health:**

* **Winter (48.3%):**
   - During winter, the River Thame may experience higher flow levels due to increased rainfall, which is common during the winter months in the UK.
   - Higher river flow during winter can result from heavy rainfall events, saturated soils, and runoff from urban and agricultural areas.
   - Increased river flow in winter may lead to the mobilization of pollutants from land surfaces and drainage systems into the river, affecting water quality.

* **Spring (24.6%):**
   - Spring in the UK sees moderate river flow levels as a result of continued rainfall and melting of winter snow in upland areas.
   - Higher river flow during spring can help replenish groundwater levels, support aquatic habitats, and maintain ecosystem functions.
   - Spring runoff may transport sediment, nutrients, and pollutants from agricultural areas and urban runoff into the river, influencing water quality.

*  **Autumn (17.5%):**
   - Autumn in the UK typically experiences decreasing river flow levels as precipitation becomes less frequent and vegetation enters dormancy.
   - Lower river flow during autumn may result in reduced dilution of pollutants and increased concentration of contaminants in the river water.
   - Autumnal leaf fall and decaying organic matter may contribute to nutrient loading and organic pollution in the river, impacting water quality.

* **Summer (9.6%):**
   - Summer in the UK often sees the lowest river flow levels due to reduced rainfall, higher temperatures, and increased evaporation.
   - Low river flow during summer can lead to water scarcity issues, particularly in areas with high water demand for agriculture and domestic use.
   - Summer drought conditions may stress aquatic ecosystems, reduce habitat availability for fish and other species, and exacerbate water quality issues such as algal blooms and nutrient enrichment.

In **summary**, while snowfall may not be a significant factor affecting river flow levels in the UK, seasonal variations in rainfall and temperature still play a crucial role in influencing water flow dynamics and water quality in rivers like the Thame. Understanding these seasonal patterns is essential for managing and protecting river ecosystems and ensuring sustainable water resource management in the region.

### Relationship between River Flow and Rain

* Now in the next visual we want to see if rain has any influence on the flow of the river.

In [ ]:
# Filter the dataframe for the Wheatley area and all dates
wheatley_filtered = rfl_wide

# Filter the dataframe for the Aylesbury area and dates from 2022 to current date
Wheatley_filtered = rain_wide
# Create line charts for both datasets
fig = px.line()

# Add line for wheatley_mean_river_flow_level
fig.add_scatter(x=wheatley_filtered['date'], y=wheatley_filtered['wheatley_mean_river_flow_level'], 
                mode='lines', name='Wheatley River Flow Level')

# Add line for wheatley_total_rainfall
fig.add_scatter(x=Wheatley_filtered['date'], y=Wheatley_filtered['wheatley_total_rainfall'], 
                mode='lines', name='Wheately Rainfall', line_color='#7DA0CA')

# Customize layout
fig.update_layout(
    title='Comparison of River Flow Level and Rainfall',
    xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d'),
    yaxis=dict(title='Value'),
    hovermode='x unified',
    xaxis_rangeslider_visible=True,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Show the plot
fig.show()


**Seasonal Impact on River Flow:**

* **June to November:** River flow remains largely unaffected by rainfall.
November to May: River flow aligns closely with rainfall, showing significant influence.

* **River Health Implications:**
    - Dry Months (June to November):  Reduced water levels and stress on aquatic ecosystems.
     - Wetter Months (November to May): Higher river flow risks erosion, sedimentation, and water quality changes.
  
* **Importance for River Management:**
    - Understanding rainfall and river flow dynamics is crucial for assessing and managing river health.
    Enables informed decisions to sustain ecological balance.

### 3.5. RTCT/ Citizen Scientist Data

* Lets look at how the dataset looks. Maybe the data from the Citizen Data might also help us get more insight into the enviroments we are working on ant the behavour of our determinants. Firstly we need to view the data.

In [ ]:
nan_count = rtct_data.isna().sum()
print(nan_count)

* The data seems to look at a lot of determinants, most of we may not be able to use and we will then not be working with some columns. But for now we are going to look at the locations that the Scientist used to take samples and decide fromhere if we can use all the data or focus on specific locations. 

In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[51.762056, -0.975278], zoom_start=10)

# Create feature groups for each dataset
tw_da_group = folium.FeatureGroup(name='tw_da_data (Red)')
wq_wide_group = folium.FeatureGroup(name='wq_wide (Blue)')
rtct_group = folium.FeatureGroup(name='rtct_data (Green)')

# Define sampling sites with different colors
special_sites = ['TH-PTAE0064', 'TH-PTAE009', 'TH-PTAE0021', 'TH-PTAE0223', 'TH-PTAE0212',
                 'TH-PTAE0024', 'TH-PTAR0028', 'TH-PTAR0020', 'TH-PTAR0021', 'TH-PTAR0048',
                 'TH-PTAR0112', 'TH-PTAR0132', 'TH-PTAR0030', 'TH-PTAR0100', 'TH-PTAR0052',
                 'TH-PTAR0022', 'TH-RSN0914']

# Create a set to store unique special sites
unique_special_sites = set()

# Filter wq_wide data to include only unique special_sites
for index, row in wq_wide.iterrows():
    if row['sampling_point_notation'] in special_sites and row['sampling_point_notation'] not in unique_special_sites:
        popup = folium.Popup(row['sampling_point_notation'], parse_html=True)
        folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='darkblue')).add_to(wq_wide_group)
        unique_special_sites.add(row['sampling_point_notation'])

# Filter the DataFrame to include only unique discharge points
unique_discharge_points = tw_da_data[tw_da_data['AlertType'].isin(['Start', 'Stop'])].drop_duplicates(subset=['latitude', 'longitude'])

# Add markers for each unique discharge point
for index, row in unique_discharge_points.iterrows():
    # Create a popup with the alert type and datetime
    popup = folium.Popup(f"Alert Type: {row['LocationName']}, DateTime: {row['DateTime']}", parse_html=True)
    
    # Add marker with red color for discharge points
    folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='red')).add_to(tw_da_group)

# Define specified site names for rtct_data
rtct_specified_sites = [
    'Lower Thame at Chippinghurst', 'Mid Thame below Long Crendon STW', 'Upper Thame ALL',
    'Upper Thame Below Tring STW', 'Lower Thame at Ickford', 'Lower Thame at Shabbington',
    'Lower Thame below Wheatley STW', 'Lower Thame ALL', 'Cuddington bridge',
    'Nether Winchendon Footbridge', 'Mid Thame below cuddington STW',
    'Lower Thame below G Milton STW', 'Lower Thame Dorchester'
]

# Filter rtct_data based on the unique specified site names
filtered_rtct_data = rtct_data[rtct_data['Site Name'].isin(rtct_specified_sites)]

# Convert rtct_specified_sites to a set to remove duplicates
rtct_specified_sites_set = set(rtct_specified_sites)

# Create a dictionary to store main land use within 50m for each site
land_use_dict = dict(zip(rtct_data['Site Name'], rtct_data['What is the main land use within 50m?']))

# Iterate over filtered_rtct_data to add markers for unique site names
for index, row in filtered_rtct_data.iterrows():
    # Check if the Site Name is in the set of unique specified sites
    if row['Site Name'] in rtct_specified_sites_set:
        # Get the main land use within 50m for the site
        land_use = land_use_dict.get(row['Site Name'], 'Unknown')
        # Create popup content with site name and land use
        popup_content = f"{row['Site Name']}Main land use within 50m: {land_use}"
        folium.Marker(
            [row['y'], row['x']],
            icon=folium.Icon(color='green'),
            popup=folium.Popup(popup_content, parse_html=True)
        ).add_to(rtct_group)

        # Remove the site name from the set to ensure only unique markers are added
        rtct_specified_sites_set.remove(row['Site Name'])

# Add feature groups to the map
wq_wide_group.add_to(m)
tw_da_group.add_to(m)
rtct_group.add_to(m)

# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Display the map
display(m)


* **Sampling Points Distribution:** 
  Shows the locations of water quality points (dark blue), discharge alerts (red), and river flow monitoring sites (green).
  
* **Special Sampling Sites:** Highlights critical water quality sites for detailed analysis.

* **Discharge Alerts:** Indicates where and when discharges occur, aiding in understanding their impact on water quality.

* **Land Use Information:** Displays the main land use within 50m of river flow sites, helping to identify potential pollution sources.

* **Geographical Context:** The Thame catchment boundary overlay provides a spatial reference for the monitoring efforts.

**Overall**, the map helps identify environmental hotspots, assess the impact of land use, understand discharge patterns, and prioritize resource allocation for effective environmental management.





a id="five"></a>

## 4. Feature Engineering

<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description Feature engineering ⚡                                                                                        |
| :------------------------------------------------------------------------------------------------------------------------- |
| In this section added new features- as identified in the EDA phase. |

---


In the upcoming feature engineering phase, we'll focus on refining our dataset to better capture the underlying patterns and relationships relevant to our problem. This involves extracting key information, transforming features for better representation, creating new features to enrich our dataset, and selecting the most informative variables to use on the POC. Through this process, we aim to enhance and improve interpretability, and ultimately achieve more accurate and reliable predictions of what is cousing our rivers to be unhealthy. Now due to some limitations we have decided to work with data from the year 2022 because some data doesnt go all the way to the year 2000, this will also help use work with less data and will help us give more current information on our platfom.

### 4.1. Rainfall Analysis

* We're going to compare the rainfall at three places: Aylesbury, Dancers End, and Wheatley. We only have data for these spots, but we want to understand rainfall across the whole area. So, we'll start by plotting the rainfall data we have for these three places on a graph. This will help us see if there are any differences in how much rain each place gets over time. We'll use different colors for each place's data, making it easier to compare them.

In [ ]:
# Convert the 'date' column to datetime if it's not already in datetime format
rain_wide['date'] = pd.to_datetime(rain_wide['date'])

# Filter the DataFrame to include only data from 2022 to 2024
rain_wide = rain_wide[(rain_wide['date'].dt.year >= 2022) & (rain_wide['date'].dt.year <= 2024)]

In [ ]:
rain_wide.shape

In [ ]:
rain_wide.head()

In [ ]:
# Filter the dataframe for the Aylesbury area and all dates
Aylesbury_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Filter the dataframe for the Wheatley area and all dates
Wheatley_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Filter the dataframe for the Dancers End area and all dates
Dancers_End_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Create a Plotly figure
fig = go.Figure()

# Add line chart trace for Aylesbury rainfall
fig.add_trace(go.Scatter(
    x=Aylesbury_filtered_line['date'],
    y=Aylesbury_filtered_line['aylesbury_total_rainfall'],
    mode='lines',
    name='Aylesbury Rainfall',
    line=dict(color='#7DA0CA')
))

# Add line chart trace for Wheatley rainfall
fig.add_trace(go.Scatter(
    x=Wheatley_filtered_line['date'],
    y=Wheatley_filtered_line['wheatley_total_rainfall'],
    mode='lines',
    name='Wheatley Rainfall',
    line=dict(color='#BBA9FF')
))

# Add line chart trace for Dancers End rainfall
fig.add_trace(go.Scatter(
    x=Dancers_End_filtered_line['date'],
    y=Dancers_End_filtered_line['dancers_end_total_rainfall'],
    mode='lines',
    name='Dancers End Rainfall',
    line=dict(color='#052659')
))

# Customize layout
fig.update_layout(
    title='Timeline of Rainfall for Aylesbury, Wheatley, and Dancers End',
    xaxis=dict(title='Date/Time', type='date'),
    yaxis=dict(title='Rainfall (mm)'),
    hovermode='x unified',
    xaxis_rangeslider_visible=True,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Add zoom tool to the toolbar
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the plot
fig.show()


* The visual provides a comprehensive overview of the rainfall patterns within the catchment area. It reveals a synchronized movement among the rainfall lines for Aylesbury, Wheatley, and Dancers_End, indicating a correlated pattern in rainfall occurrence across the three locations. Specifically, when rainfall occurs in Aylesbury, a similar trend is observed in Wheatley, albeit with slight variations in the amount of rainfall. Similarly, despite Dancers_End experiencing higher rainfall amounts compared to the other two locations, the visual suggests a consistent influence of rainfall across all three areas. Consequently, we have decided to retain data from all three locations and utilize the mean of their rainfall values. This approach ensures a more representative assessment of rainfall distribution within the catchment, enabling a more accurate analysis of its impact on other locations.

In [ ]:
rain_wide.head()

In [ ]:
# Calculate the mean of the three rainfall columns for each row
rain_wide['rain_mean'] = rain_wide[['aylesbury_total_rainfall', 'wheatley_total_rainfall']].mean(axis=1)

# Display the DataFrame to verify the results
print(rain_wide.head())

# If you want to take the mean for each day and create a new column 'rain_mean':
# Convert 'date' column to datetime if it's not already
rain_wide['date'] = pd.to_datetime(rain_wide['date'])

# Calculate the mean rainfall for each day
daily_rain_mean = rain_wide.groupby('date')['rain_mean'].mean().reset_index()

# Rename the column to 'rain_mean'
daily_rain_mean = daily_rain_mean.rename(columns={'rain_mean': 'rain_mean'})

# Keep only the necessary columns from daily_rain_mean DataFrame
daily_rain_mean = daily_rain_mean[['date', 'rain_mean']]

# Merge the mean rainfall data with the original DataFrame
rain_wide = pd.merge(rain_wide, daily_rain_mean, on='date', how='left')

# Drop the 'year' column
rain_wide.drop(columns=['year'], inplace=True)

In [ ]:
# Drop one of the 'rain_mean' columns
rain_wide = rain_wide.drop(columns=['rain_mean_y'])

# Verify the DataFrame after dropping the column
print(rain_wide.head())


In [ ]:
# Filter the dataframe for the Aylesbury area and all dates
Aylesbury_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Filter the dataframe for the Wheatley area and all dates
Wheatley_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Filter the dataframe for the Dancers End area and all dates
Dancers_End_filtered_line = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Create a Plotly figure
fig = go.Figure()

# Add line chart trace for Aylesbury rainfall
fig.add_trace(go.Scatter(
    x=Aylesbury_filtered_line['date'],
    y=Aylesbury_filtered_line['aylesbury_total_rainfall'],
    mode='lines',
    name='Aylesbury Rainfall',
    line=dict(color='#7DA0CA')
))

# Add line chart trace for Wheatley rainfall
fig.add_trace(go.Scatter(
    x=Wheatley_filtered_line['date'],
    y=Wheatley_filtered_line['wheatley_total_rainfall'],
    mode='lines',
    name='Wheatley Rainfall',
    line=dict(color='#380F66')
))


# Add line chart trace for the mean rainfall
fig.add_trace(go.Scatter(
    x=Aylesbury_filtered_line['date'],
    y=Aylesbury_filtered_line['rain_mean_x'],
    mode='lines',
    name='Mean Rainfall',
    line=dict(color='green')  # Change color to green
))

# Customize layout
fig.update_layout(
    title='Timeline of Rainfall for Aylesbury  and  Wheatley',
    xaxis=dict(title='Date/Time', type='date'),
    yaxis=dict(title='Rainfall (mm)'),
    hovermode='x unified',
    xaxis_rangeslider_visible=True,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Add zoom tool to the toolbar
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the plot
fig.show()


* The visualization of the rainfall data indicates a consistent pattern across the three locations – Aylesbury, Dancers End, and Wheatley. Despite slight variations in rainfall levels at each location, the mean rainfall calculated from these datasets appears to align closely with the individual rainfall trends. This alignment suggests that the mean derived from the three datasets is reliable and representative of the overall rainfall pattern within the catchment area. Consequently, moving forward, we intend to utilize this mean rainfall value as a standard reference for analyzing data from all sampling points within the catchment area, facilitating a comprehensive analysis through facet plots.

### 4.2. River Flow

For the river flow data we have a small issue, we will only be using the Wheatly River Flow data for the entire catchment since its the only location that measures river flow in the catchment. 

River flow is not typically constant across the entire river stretch. Several factors contribute to variations in river flow, including topography, weather patterns, seasonal changes, human activities, and the presence of tributaries or diversions along the river course. For instance, in mountainous regions, river flow may vary significantly due to differences in elevation and precipitation patterns. Additionally, human interventions such as dams, irrigation withdrawals, and wastewater discharges can alter river flow patterns. Therefore, while some sections of a river may exhibit relatively stable flow rates, others may experience fluctuations depending on the prevailing conditions and anthropogenic influences.

Applying river flow data from a single location to all sampling points along the river can be a useful approximation, especially when detailed flow measurements are not available for each sampling point. However, it's essential to recognize that river flow can vary spatially due to factors such as channel morphology, land use changes, and local hydrological conditions.

Using river flow data from a single location assumes that flow conditions are relatively uniform along the river reach. While this approach can provide a simplified representation of flow dynamics, it may overlook localized variations that could influence water quality parameters at specific sampling points.

Therefore, while applying a single river flow value to all sampling points can offer insights into general trends, it's crucial to interpret the results cautiously and consider the potential limitations of this approach. Integrating additional data sources or conducting site-specific measurements where feasible can enhance the accuracy and reliability of the analysis.

In [ ]:
# Convert the 'date' column to datetime if it's not already in datetime format
rfl_wide['date'] = pd.to_datetime(rfl_wide['date'])

# Filter the DataFrame to include only data from 2022 to 2024
rfl_wide = rfl_wide[(rfl_wide['date'].dt.year >= 2022) & (rfl_wide['date'].dt.year <= 2024)]

In [ ]:
rfl_wide.shape

In [ ]:
rfl_wide.head()

In [ ]:
# Rename the column 'wheatley_mean_river_flow_level' to 'river_thame_river_flow_level'
rfl_wide.rename(columns={'wheatley_mean_river_flow_level': 'river_thame_river_flow_level'}, inplace=True)

# Display the DataFrame after renaming the column
print(rfl_wide.head())

#### River Thame Flow Level

In [ ]:
# Filter the dataframe for the River Thame area and all dates
wheatley_filtered = rfl_wide

# Create line chart for the river_thame_mean_river_flow_level
fig = go.Figure()

# Add line for river_thame_mean_river_flow_level
fig.add_scatter(x=wheatley_filtered['date'], y=wheatley_filtered['river_thame_river_flow_level'], 
                mode='lines', name='River Thame Flow Level')

# Customize layout
fig.update_layout(
    title='River Thame Flow Level',
    xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d'),
    yaxis=dict(title='Flow Level'),
    hovermode='x unified',
    xaxis_rangeslider_visible=True,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Show the plot
fig.show()

* Moving forward, we will incorporate this chart along with the rest of the visuals into a facet plot. This will allow us to examine whether river flow impacts water quality, in conjunction with the Environmental Data Metrics (EDMs).

### 4.3. Discharge Points

* Now, we aim to observe the discharges on each individual EDM.

In [ ]:
# Filter the DataFrame for specific columns including 'X' and 'Y'
tw_da_data_edm = tw_da_data[[ 'LocationName', 'AlertType', 'DateTime']]

# Convert the 'DateTime' column to datetime objects
tw_da_data_edm['DateTime'] = pd.to_datetime(tw_da_data['DateTime'])

# Replace string values in the 'AlertType' column
tw_da_data['AlertType'] = tw_da_data['AlertType'].replace({'Start': 34, 'Offline start': 0, 'Stop': 0, 'Offline stop': 0})

# Display the DataFrame
print(tw_da_data_edm.head())

In [ ]:
tw_da_data_edm.shape

In [ ]:
# List of location names you want to filter by
locations = ['Cuddington', 'Dorchester', 'Aylesbury', 'Marsworth', 'Haddenham', 'Long Crendon', 
             'Shabbington', 'Stone', 'Thame', 'Wingrave', 'Stewkley', 'Tring', 'Rowsham', 
             'Waddesdon', 'Princes Risborough', 'Chinnor', 'Dorton', 'Tetsworth', 'Watlington', 
             'Chalgrove', 'Little Milton', 'Horton-Cum-Studley', 'Forest Hill', 'Wheatley']

# Loop through each location and create a separate plot
for location in locations:
    # Filter the dataframe for the current location
    location_data = tw_da_data_edm[tw_da_data_edm['LocationName'] == location]
    
    # Replace values above 34 with 34 in the 'AlertType' column
    location_data['AlertType'] = location_data['AlertType'].apply(lambda x: 34 if x > 34 else x)
    
    # Plot the data for the current location
    fig = go.Figure()

    # Add bar chart trace for AlertType
    fig.add_trace(go.Bar(
        x=location_data['DateTime'],
        y=location_data['AlertType'],
        name='',
        marker_color='#673AB7',
        marker_line_color='#673AB7',  # Set bar outline color
        marker_line_width=1,  # Set bar outline width
        width=8  # Adjust the width of the bars
    ))

    # Customize layout
    fig.update_layout(
        title=f'Timeline of Discharge for EDM - {location}',
        xaxis=dict(title='Date/Time', type='date', range=['2022-01-01', '2024-04-30']),
        yaxis=dict(title=''),
        hovermode='x unified',
        xaxis_rangeslider_visible=True,
        plot_bgcolor='#B2DFDB'  # Background color
    )

    # Set the y-axis range to 0-34
    fig.update_yaxes(range=[0, 34])

    # Add zoom tool to the toolbar
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    # Show the plot
    fig.show()


* We will use these visuals to analyze the relationship between discharges and the corresponding determinants at various sampling points along the river. Each discharge point will be linked to its respective sampling point, allowing us to comprehensively understand how these discharges influence the determinants at each location.

### 4.4. Determinants

* Now we will create the features to be used in the app. First, we will filter the data to include only records from the year 2022 to the present. Next, we will focus on specific determinants and exclude others. Finally, we will link the sampling points to the EDMs, rainfall, and flow data.

#### 4.4.1. Firstly we will reduce our data and look at it from the date 2022 to date.

In [ ]:
wq_wide.head()

In [ ]:
# Convert the 'sample_date_time' column to datetime format
wq_wide['sample_date_time'] = pd.to_datetime(wq_wide['sample_date_time'])

# Filter data from 2022 onwards
filtered_data = wq_wide[wq_wide['sample_date_time'] >= '2022-01-01']


In [ ]:
filtered_data.shape

* Now the next focus should be the Sampling Points along the River.

In [ ]:
# List of special sites
special_sites = ['TH-PTAE0064', 'TH-PTAE009', 'TH-PTAE0021', 'TH-PTAE0223', 'TH-PTAE0212',
                 'TH-PTAE0024', 'TH-PTAR0028', 'TH-PTAR0020', 'TH-PTAR0021', 'TH-PTAR0048',
                 'TH-PTAR0112', 'TH-PTAR0132', 'TH-PTAR0030', 'TH-PTAR0100', 'TH-PTAR0052',
                 'TH-PTAR0022', 'TH-RSN0914']

# Filter the dataframe to include only the special sites
filtered_data = filtered_data[filtered_data['sampling_point_notation'].isin(special_sites)]


In [ ]:
filtered_data.shape

* It's apparent that our dataset contains fewer records than anticipated. Now, we need to verify whether each sampling point from the river is still represented in the updated dataset. It's conceivable that certain sampling points may lack the most recent data.

In [ ]:
# Count occurrences of each sampling point from special_sites in the filtered dataframe
sampling_point_counts = filtered_data['sampling_point_notation'].value_counts()

# Display the counts
print(sampling_point_counts)


We wanted to examine the counts of each sampling point in the filtered dataset to understand the distribution of data across different sampling points. This interest stemmed from noticing a reduction in the amount of data after filtering the dataset. By calculating the frequency of occurrence for each sampling point, We could gauge the representation of each location in the reduced dataset. The count of occurrences for each sampling point provides valuable insights into the availability and coverage of data. For example, sampling points with higher counts are more frequently observed in the dataset, suggesting a more comprehensive data collection effort or higher monitoring frequency at those locations. Conversely, sampling points with lower counts may indicate a lack of recent data or less frequent sampling. Understanding these counts is crucial for assessing the reliability and completeness of the dataset for further analysis and decision-making processes.

Now from this results we can see that the amount of times the data was taken is too low an will make it hard for us te get the exact relationship of effects the other factors such as EDMs, Rain and Flow have an effect on the determinants along the sampling points on the river. So one recormandation we can make is that we need to have samples taken on a more ragular basis than e.g. 28 times in a period of two years.

**Now we want to see a new geospacial view of the sampling points.**

TH-PTAR0030, TH-PTAR0028, TH-PTAR0022, TH-PTAE0024, TH-PTAR0048, TH-PTAR0021, TH-PTAR0020, TH-PTAE0064, TH-PTAE0021, TH-PTAR0112, TH-RSN0914.


In [ ]:
# Define sampling sites with different colors
special_sites = ['TH-PTAR0030','TH-PTAR0028', 'TH-PTAR0022', 'TH-PTAE0024', 'TH-PTAR0048', 'TH-PTAR0021', 'TH-PTAR0020', 'TH-PTAE0064', 'TH-PTAE0021', 'TH-PTAR0112', 'TH-RSN0914']

filtered_wq = wq_wide[wq_wide['sampling_point_notation'].isin(special_sites)]

print(filtered_wq.shape)

In [ ]:
# Create a set to store unique coordinates
unique_coordinates = set()

# Create a map
m = folium.Map(location=[wq_wide['latitude'].mean(), wq_wide['longitude'].mean()], zoom_start=10)

# Add markers for each unique latitude and longitude point
for index, row in filtered_wq.iterrows():
    # Check if the coordinates are unique
    if (row['latitude'], row['longitude']) not in unique_coordinates:
        # Create a popup with the sampling_point_notation
        popup = folium.Popup(row['sampling_point_notation'], parse_html=True)
        
        # Check if the sampling point is special
        if row['sampling_point_notation'] in special_sites:
            # Add marker with custom color
            folium.Marker([row['latitude'], row['longitude']], popup=popup, icon=folium.Icon(color='darkblue')).add_to(m)
        else:
            # Add marker with default color
            folium.Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)
        
        # Add the coordinates to the set
        unique_coordinates.add((row['latitude'], row['longitude']))


# add thame catchment geojson data
geojson_data='../src/thame_catchment.geojson'
folium.GeoJson(data=geojson_data).add_to(m)

# Display the map
m

#### 4.4.2. Dropping Unnecessary Determinants
* From our analysis we need to drop some determinants that do not affect the data and do not play much of a role in the overrall Health of the River.

In [ ]:
# Specify the columns to be dropped
columns_to_drop = ['easting', 'northing', 'conductivity', 'iron', 'nitrite', 
                   'dissolved_oxygen_%_saturation', 'Phosphate']

# Drop the specified columns from the dataset
filtered_data = filtered_data.drop(columns=columns_to_drop)


In [ ]:
filtered_data.shape

In [ ]:
filtered_data.head()

Now that we have reduced the dataset and removed some sampling points, we aim to establish thresholds to determine whether each water quality determinant is within acceptable limits, above, or below. To accomplish this, we referenced information from the Environment Agency (EA), as outlined in the document [provided link](https://www.legislation.gov.uk/uksi/2015/1623/pdfs/uksiod_20151623_en_003.pdf). Additionally, for certain determinants, we relied on international standards, considering the potential for this product to have global applicability. By incorporating these thresholds, we can effectively evaluate water quality parameters and identify instances where they deviate from established standards, ensuring comprehensive monitoring and assessment of environmental conditions.

#### 4.4.3. Threshold values for determinants

In [ ]:
# Define threshold values for each determinant
thresholds = {
    'ammoniacal_nitrogen': {'min': 0, 'max': 1},
    'bod_atu': {'min': 0, 'max': 4},
    'ph': {'min': 6.5, 'max': 8.5},
    'nitrogen_oxidised': {'min': 0, 'max': 2},
    'nitrate': {'min': 0, 'max': 10},
    'orthophosphate': {'min': 0, 'max': 0.1},
    'water_temperature_in_celsius': {'min': 10, 'max': 25},
    'dissolved_oxygen_O2': {'min': 5, 'max': 10},
    'unionised_ammonia': {'min': 0, 'max': 0.5},
    'solids_suspended_at_105C': {'min': 0, 'max': 50}
}

# Initialize a list to store the counts of determinants out of limits
out_of_limits_count = []

# Iterate through each row in the dataframe
for index, row in filtered_data.iterrows():
    # Initialize count for determinants out of limits
    count_out_of_limits = 0
    
    # Iterate through each determinant column and check if it's within limits
    for determinant, threshold in thresholds.items():
        value = row[determinant]
        if value < threshold['min'] or value > threshold['max']:
            count_out_of_limits += 1
    
    # Append the count of determinants out of limits for the current row
    out_of_limits_count.append(count_out_of_limits)

# Add the out of limits count to the dataframe as a new column
filtered_data['Out of Limits'] = out_of_limits_count


In [ ]:
filtered_data.head()

#### Facet Plot

The next thing we would like to see now is the visuals aligned in a row and having them in this order.

##### Determinants
- "ammoniacal_nitrogen"
- "bod_atu"
- "solids_suspended_at_105C"
- "nitrogen_oxidised"
- "nitrate"
- "orthophosphate"
- "dissolved_oxygen_O2"

##### EDMs (Discharge Points)
The EDMs that directly affect or flow into the sampling point.

##### River Flow
Using one set of data from Wheatley.

##### Rainfall
Using the Mean rainfall data for the catchment.


In [ ]:
# Threshold values for determinants with units
thresholds = {
    'ammoniacal_nitrogen': {'min': 0, 'max': 1, 'unit': 'mg/l'},
    'bod_atu': {'min': 0, 'max': 4, 'unit': 'mg/l'},
    'ph': {'min': 6.5, 'max': 8.5, 'unit': 'phunits'},
    'nitrogen_oxidised': {'min': 0, 'max': 2, 'unit': 'mg/l'},
    'nitrate': {'min': 0, 'max': 10, 'unit': 'mg/l'},
    'orthophosphate': {'min': 0, 'max': 0.1, 'unit': 'mg/l'},
    'dissolved_oxygen_O2': {'min': 5, 'max': 10, 'unit': 'mg/l'},
    'solids_suspended_at_105C': {'min': 0, 'max': 50, 'unit': 'mg/l'}
}

# Sampling points and their labels
sampling_points = {
    'TH-PTAR0030': 'THAME AT WHEATLEY BRIDGE'
}

# Define determinants
determinants = [
    "ammoniacal_nitrogen", "bod_atu", "solids_suspended_at_105C", "nitrogen_oxidised", "nitrate", "orthophosphate", "dissolved_oxygen_O2"
]

# Filter data for the range of interest (from 2022 to current date)
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime.now()
wq_wide_filtered = wq_wide[(wq_wide['sample_date_time'] >= start_date) & (wq_wide['sample_date_time'] <= end_date)]

# Create a separate plot for each determinant
for determinant in determinants:
    # Create empty figure
    fig = px.line(title=f'Time Series Plot of {determinant} for Different Sampling Points',
                  labels={'sample_date_time': 'Date', 'value': f'Value ({thresholds[determinant]["unit"]})'},
                  template='plotly_white')

    # Add lines for each sampling point
    for sample_point, label in sampling_points.items():
        # Filter the dataframe for the current sample point
        sample_data = wq_wide_filtered[wq_wide_filtered['sampling_point_notation'] == sample_point]

        # Plot the data for the current determinant with different colors
        fig.add_scatter(x=sample_data['sample_date_time'], y=sample_data[determinant],
                        mode='lines', name=label)

    # Add threshold lines
    fig.add_hline(y=thresholds[determinant]["min"], line_dash="dash", line_color="red",
                  annotation_text=f"Lower Limit ({thresholds[determinant]['min']} {thresholds[determinant]['unit']})", 
                  annotation_position="bottom right")
    fig.add_hline(y=thresholds[determinant]["max"], line_dash="dash", line_color="red",
                  annotation_text=f"Upper Limit ({thresholds[determinant]['max']} {thresholds[determinant]['unit']})", 
                  annotation_position="top right")

    # Customize layout
    fig.update_layout(
        xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d'),
        yaxis=dict(title=f'Value ({thresholds[determinant]["unit"]})'),
        hovermode='x unified',
        xaxis_rangeslider_visible=False,
        plot_bgcolor='#B2DFDB'  # Set background color
    )

    # Show the plot
    fig.show()

# List of location names you want to filter by
locations = ['Worminghall', 'Forest Hill', 'Wheatley']

 # EDM Locations linked to each sampling point

 #    'TH-PTAE0064': ['Wingrave', 'Rowsham'],
 #    'TH-PTAR0028': ['Stewkley'],
 #    'TH-PTAR0020': ['Aylesbury', 'Waddesdon'],
 #    'TH-PTAE0021': ['Cuddington'],
 #    'TH-PTAR0021': ['Cuddington'],
 #    'TH-PTAR0048': ['Dorton'],
 #    'TH-PTAR0112': ['Haddenham', 'Thame'],
 #    'TH-PTAR0030': ['Worminghall', 'Forest Hill', 'Wheatley'],
 #    'TH-RSM0914': ['Wheatley', 'Little Milton'],
 #    'TH-PTAR0022': ['Wheatley', 'Little Milton],
 #    'TH-PTAE0024': ['Dorchester']

# Loop through each location and create a separate plot
for location in locations:
    # Filter the dataframe for the current location
    location_data = tw_da_data_edm[tw_da_data_edm['LocationName'] == location]
    
    # Replace values above 34 with 34 in the 'AlertType' column
    location_data['AlertType'] = location_data['AlertType'].apply(lambda x: 34 if x > 34 else x)
    
    # Plot the data for the current location
    fig = go.Figure()

    # Add bar chart trace for AlertType
    fig.add_trace(go.Bar(
        x=location_data['DateTime'],
        y=location_data['AlertType'],
        name='',
        marker_color='#673AB7',
        marker_line_color='#673AB7',  # Set bar outline color
        marker_line_width=1,  # Set bar outline width
        width=8  # Adjust the width of the bars
    ))

    # Customize layout
    fig.update_layout(
        title=f'Timeline of Events for EDM - {location}',
        xaxis=dict(title='Date/Time', type='date', range=['2022-01-01', '2024-04-30']),
        yaxis=dict(title=''),
        hovermode='x unified',
        xaxis_rangeslider_visible=False,
        plot_bgcolor='#B2DFDB'  # Background color
    )

    # Set the y-axis range to 0-34
    fig.update_yaxes(range=[0, 34])

    # Add zoom tool to the toolbar
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=False),
            type="date"
        )
    )

    # Show the plot
    fig.show()

# Filter the dataframe for the River Thame area and all dates
wheatley_filtered = rfl_wide

# Create line chart for the river_thame_mean_river_flow_level
fig = go.Figure()

# Add line for river_thame_mean_river_flow_level
fig.add_scatter(x=wheatley_filtered['date'], y=wheatley_filtered['river_thame_river_flow_level'], 
                mode='lines', name='River Thame Flow Level')

# Customize layout
fig.update_layout(
    title='River Thame Flow Level',
    xaxis=dict(title='Date', type='date', tickformat='%Y-%m-%d'),
    yaxis=dict(title='Flow Level'),
    hovermode='x unified',
    xaxis_rangeslider_visible=False,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Show the plot
fig.show()

# Filter the dataframe for the Aylesbury and Wheatley areas and all dates
filtered_rainfall_data = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

# Create a Plotly figure
fig = go.Figure()

# Add line chart trace for the mean rainfall
fig.add_trace(go.Scatter(
    x=filtered_rainfall_data['date'],
    y=filtered_rainfall_data['rain_mean_x'],
    mode='lines',
    name='Mean Rainfall',
    line=dict(color='green')  # Change color to green
))

# Customize layout
fig.update_layout(
    title='Timeline of Rainfall',
    xaxis=dict(title='Date/Time', type='date'),
    yaxis=dict(title='Rainfall (mm)'),
    hovermode='x unified',
    xaxis_rangeslider_visible=False,
    plot_bgcolor='#B2DFDB'  # Background color
)

# Add zoom tool to the toolbar
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    showlegend=False  # Remove the legend
)

# Show the plot
fig.show()

* On this visual, we've achieved a neat alignment of the determinants. This arrangement aids in understanding the influences of each factor. We observe that a significant portion of discharges correlates with rainfall and high river flow. Additionally, some determinants appear to be influenced not only by river flow but also by discharges. We intend to incorporate this feature into the app, allowing clients to visualize the interplay between different factors and understand their effects better.

<a id="nine"></a>

## 5. Conclusions

<a href=#cont>Back to Table of Contents</a>

---

| ⚡ Description: Conclusions ⚡                                                                                                    |
| :--------------------------------------------------------------------------------------------------------------------------------------- |
| In this section we concluded the project. |

---


The insights gleaned from the data provide a nuanced understanding of how factors such as sewage discharges, agricultural runoff, and industrial waste directly influence water quality. By examining trends and correlations within the data, users can identify specific indicators that serve as reliable markers for assessing the health of rivers. These indicators may include nutrient levels, presence of pollutants, microbial contamination, and overall biodiversity.

Moreover, these insights offer a holistic view of the complex interplay between human activities and natural processes, shedding light on the broader implications for river ecosystems. Armed with this knowledge, stakeholders can develop targeted strategies and policies to mitigate the negative impacts of these factors and restore balance to river ecosystems. Ultimately, these insights serve as a valuable resource for guiding decision-making and fostering sustainable practices that safeguard the health and integrity of our rivers for future generations.

Our findings indicate that rivers predominantly receive water from rainfall, with its influence on river flow varying throughout different seasons. Subsequently, heightened rainfall and increased river flow often coincide with the discharge of waste by sewage treatment facilities into rivers. This introduces contaminants, leading to spikes in certain water quality indicators.

However, our analysis encountered a challenge. The irregular sampling frequency of water quality data, unlike the consistent monitoring of rainfall and river flow, complicates the attribution of spikes in water quality indicators solely to rainfall, flow, or sewage discharge.

Furthermore, examination of data contributed by citizen scientists and the assessment of surrounding land use patterns suggest that runoff from agricultural and urban areas may also contribute to water contamination. This is particularly significant considering the limited availability of sewage discharge data, spanning from late 2023 to early 2024. Consequently, it is plausible that factors other than untreated sewage contributed to fluctuations in water quality indicators prior to this timeframe.

We think that further investigation is warranted. By extending the duration of data collection and analysis, we can achieve a more comprehensive understanding of the factors influencing river health.But for the purpose of the POC this is the work we have done.

####  Export preprocessed water quality data to S3 streamlit folder

In [ ]:
# WATER QUALITY EXPORT TO S3 STREAMLIT FOLDER

key = "datasets/streamlit_data/streamlit_water_quality_df.csv"  # Construct the S3 object key
csv_buffer = filtered_data.to_csv(None, index=False)  # Generate CSV data as a string
s3_client.put_object(Body=csv_buffer, Bucket=S3_BUCKET_NAME, Key=key)  # Upload to S3

#### Export preprocessed rainfall and river flow data to S3 streamlit folder

In [ ]:
# RAINFALL EXPORT TO S3 STREAMLIT FOLDER

# Filter the data for specific dates
filtered_rainfall_data = rain_wide[(rain_wide['date'] >= '2022-01-01') & (rain_wide['date'] <= pd.Timestamp.today())]

rain_key = "datasets/streamlit_data/streamlit_rainfall_df.csv"  # Construct the S3 object key
rain_csv_buffer = filtered_rainfall_data.to_csv(None, index=False)  # Generate CSV data as a string
s3_client.put_object(Body=rain_csv_buffer, Bucket=S3_BUCKET_NAME, Key=rain_key)  # Upload to S3


In [ ]:
# RIVER FLOW EXPORT TO S3 STREAMLIT FOLDER

rfl_key = "datasets/streamlit_data/streamlit_river_flow_df.csv"  # Construct the S3 object key
rfl_csv_buffer = rfl_wide.to_csv(None, index=False)  # Generate CSV data as a string
s3_client.put_object(Body=rfl_csv_buffer, Bucket=S3_BUCKET_NAME, Key=rfl_key)  # Upload to S3